In [1]:
import tensorflow as tf
import datetime
import numpy as np
import csv

In [3]:
!pip3 install bert-tensorflow

In [2]:
import bert
#from bert import run_classifier
from bert import optimization
from bert import modeling
#from bert import tokenization

W0419 15:36:43.464694 140736186118976 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [3]:
OUTPUT_DIR = '../output'


# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
NUM_TRAIN_STEPS = 3
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 50
SAVE_SUMMARY_STEPS = 10

In [4]:
def gather_indexes(sequence_tensor, positions):
    """Gathers the vectors at the specific positions over a minibatch."""
    sequence_shape = modeling.get_shape_list(sequence_tensor, expected_rank=3)
    
    batch_size = sequence_shape[0]
    seq_length = sequence_shape[1]
    width = sequence_shape[2]
    
    flat_offsets = tf.reshape(
        tf.range(0, batch_size, dtype=tf.int32) * seq_length, [-1, 1])
    flat_positions = tf.reshape(positions + flat_offsets, [-1])
    flat_sequence_tensor = tf.reshape(sequence_tensor,
                                    [batch_size * seq_length, width])
    output_tensor = tf.gather(flat_sequence_tensor, flat_positions)
    return output_tensor

In [5]:
def get_lm_output(bert_config, input_tensor, output_weights, positions,
                         label_ids, label_weights):
    """Get loss and log probs for the masked LM."""
    input_tensor = gather_indexes(input_tensor, positions)
    
    with tf.variable_scope("cls/predictions"):
        # We apply one more non-linear transformation before the output layer.
        # This matrix is not used after pre-training.
        with tf.variable_scope("transform"):
            input_tensor = tf.layers.dense(
                input_tensor,
                units=bert_config.hidden_size,
                activation=modeling.get_activation(bert_config.hidden_act),
                kernel_initializer=modeling.create_initializer(
                    bert_config.initializer_range))
            input_tensor = modeling.layer_norm(input_tensor)

        # The output weights are the same as the input embeddings, but there is
        # an output-only bias for each token.
        output_bias = tf.get_variable(
            "output_bias",
            shape=[bert_config.vocab_size],
            initializer=tf.zeros_initializer())
        logits = tf.matmul(input_tensor, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        
        label_ids = tf.reshape(label_ids, [-1])
        label_weights = tf.reshape(label_weights, [-1])

        one_hot_labels = tf.one_hot(
            label_ids, depth=bert_config.vocab_size, dtype=tf.float32)

        # The `positions` tensor might be zero-padded (if the sequence is too
        # short to have the maximum number of predictions). The `label_weights`
        # tensor has a value of 1.0 for every real prediction and 0.0 for the
        # padding predictions.
        per_example_loss = -tf.reduce_sum(log_probs * one_hot_labels, axis=[-1])
        numerator = tf.reduce_sum(label_weights * per_example_loss)
        denominator = tf.reduce_sum(label_weights) + 1e-5
        loss = numerator / denominator
    
    return (loss, predicted_labels, log_probs)


In [6]:
def create_model(is_training, input_ids, input_mask, segment_ids, labels):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)
    
    (lm_loss, lm_predicted_labels, lm_log_probs) = get_lm_output(
         bert_config, model.get_sequence_output(), model.get_embedding_table(),
         masked_lm_positions, masked_lm_ids, masked_lm_weights)
    if is_training:
        return (lm_loss, lm_predicted_labels, lm_log_probs)
    else:
        return (lm_predicted_labels,log_probs)


In [7]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(bert_config, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""
    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
        # TRAIN and EVAL
        if not is_predicting:
        
            (loss, predicted_labels, log_probs) = create_model(
                is_training, input_ids, input_mask, segment_ids, label_ids)
        
            train_op = optimization.create_optimizer(
                loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)
        
             #Calculate evaluation metrics. 
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(
                    label_ids,
                    predicted_labels)
                auc = tf.metrics.auc(
                    label_ids,
                    predicted_labels)
                recall = tf.metrics.recall(
                    label_ids,
                    predicted_labels)
                precision = tf.metrics.precision(
                    label_ids,
                    predicted_labels) 
                true_pos = tf.metrics.true_positives(
                    label_ids,
                    predicted_labels)
                true_neg = tf.metrics.true_negatives(
                    label_ids,
                    predicted_labels)   
                false_pos = tf.metrics.false_positives(
                    label_ids,
                    predicted_labels)  
                false_neg = tf.metrics.false_negatives(
                    label_ids,
                    predicted_labels)
                return {
                    "eval_accuracy": accuracy,
                    "f1_score": f1_score,
                    "auc": auc,
                    "precision": precision,
                    "recall": recall,
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg
                }
        
            eval_metrics = metric_fn(label_ids, predicted_labels)
            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode,
                    loss=loss,
                    train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode,
                    loss=loss,
                    eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(
                is_training, input_ids, input_mask, segment_ids, label_ids)
            predictions = {
              'probabilities': log_probs,
              'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn

In [8]:
# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [9]:
bert_config = modeling.BertConfig.from_json_file('./bert_config.json')

W0419 15:36:58.858961 140736186118976 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [ ]:
def data_reader(input_file, max_seq_length, max_predictions_per_seq=1):
    # read out the raw input
    with open(input_file) as file:
        reader = csv.reader(file)
        sentences = [[int(i) for i in row] for row in reader]
    keys = ['input_ids','mask_ids','segment_ids', 'masked_lm_positions', 'masked_lm_weights']
    features = dict([(key, []) for key in keys])
    
    for row in sentences:
        length = min(len(row),max_seq_length)
        sentence = row[len(row)-length:]
        input_ids = np.pad(row,(0,max_seq_length-len(sentence)),'constant', constant_values=(0, 0))
        mask_ids = np.ones(len(sentence))
        mask_ids = np.pad(mask_ids,(0,max_seq_length-len(sentence)),'constant',constant_values=(0, 0))
        segment_ids = np.ones(max_seq_length)
        masked_lm_positions = np.array(len(sentence)-1)
        masked_lm_weights = np.array(1.0)
        
        features['input_ids'].append(input_ids)
        features['mask_ids'].append(mask_ids)
        features['segment_ids'].append(segment_ids)
        features['masked_lm_positions'].append(masked_lm_positions)
        features['masked_lm_weights'].append(masked_lm_weights)
    return features
        
        
    

In [ ]:
def input_fn_builder(feature_reader,
                     is_training):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""
    
    def input_fn(params):
        """An input function for training or evaluating"""
        batch_size = params["batch_size"]
        # Convert the inputs to a Dataset.
        dataset = tf.data.Dataset.from_tensor_slices(dict(feature_reader))

        # Shuffle and repeat if you are in training mode.
        if training:
            dataset = dataset.shuffle(1000).repeat()
    
        return dataset.batch(batch_size)

    return input_fn

In [ ]:
input_file = '../../data/tasks/dep2vec/train/dependency_traces_training_ordered.csv'
feature_reader = data_reader(input_file,10)
train_input_fn = input_fn_builder(feature_reader, is_training=True)

In [13]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(feature_reader['input_ids']) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [33]:
print(len(feature_reader['input_ids']))

456632


In [14]:
model_fn = model_fn_builder(bert_config,  
                            learning_rate=LEARNING_RATE,
                            num_train_steps=num_train_steps, 
                            num_warmup_steps=num_warmup_steps, 
                            use_tpu=False,
                            use_one_hot_embeddings=True)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


In [12]:
input_file = '../../data/tasks/dep2vec/train/dependency_traces_training_ordered.csv'
feature_reader = data_reader(input_file,10)
train_input_fn = input_fn_builder(feature_reader, is_training=True)

In [15]:
print(f'Beginning Training!')
#current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
#print("Training took time ", datetime.now() - current_time)

W0419 15:37:53.359685 140736186118976 deprecation.py:323] From /Users/arielxiao/Library/Python/3.7/lib/python/site-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Beginning Training!


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
